In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import gc
import glob
import os
import csv

import keras
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras import optimizers
from keras.models import load_model
%matplotlib inline

/home/fx/env/stock/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [7]:
SET50 = ['ADVANC', 'AOT', 'BANPU', 'BBL', 'BCP', 'BDMS',
        'BEM', 'BH', 'BJC', 'BTS', 'CENTEL', 'CPALL', 
         'CPF', 'CPN', 'DTAC', 'EGCO', 'HMPRO', 
         'INTUCH', 'IRPC', 'KBANK', 'KCE', 'KKP', 'KTB', 'LH', 'MINT', 'PTT',
        'PTTEP', 'ROBINS', 'SCB', 'SCC', 
        'TCAP', 'TISCO', 'TMB', 'TOP', 'TRUE', 'TU']

In [8]:
def load_stock_data(symbol, year_start, year_end):
    stock_data = []
    with open('../Data set/SET50_OHLC/' + symbol + '.BK.csv', 'r') as csv_file:
        file_data = csv.reader(csv_file, delimiter=',')
        file_data = list(file_data)[1:]
        ma = []
        for row in file_data:
            if row[1] is '':
                continue
            elif len(ma) < 30:
                ma.append([float(row[1]), float(row[2]), float(row[3]), float(row[4])])
            elif int(row[0][0:4]) >= year_start and int(row[0][0:4]) <= year_end:
                temp = [float(row[1]), float(row[2]), float(row[3]), float(row[4])]
                stock_data.append(find_diff_ma(ma, temp))
                del ma[0]
                ma.append(temp)
    return stock_data

def find_diff_ma(moving_averages, current_values):
    returns = []
    for i in range(len(moving_averages[0])):
        data = [moving_average[i] for moving_average in moving_averages]
        ma = np.sum(data) / 30
        diff_ma = current_values[i] - ma
        returns.append(diff_ma)
    return returns
        

In [9]:
stock_datas = [ load_stock_data(x, 2008, 2014) for x in SET50 ]

rounds = len(stock_datas[0]) - 30

In [11]:
data = []
count = 0
index = 0
while index < rounds:
    temp = [stock[index : index + 30] for stock in stock_datas]
    data.append(temp)
    index += 1
    
x = np.asarray(data)
x_train = x.astype('float32')

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], x_train.shape[2] * x_train.shape[3]))

print(x_train.shape)

(1690, 36, 120) (1, 36, 120)


In [12]:
target = []
index = 0
while index < rounds:
    temp = [stock[index + 30] for stock in stock_datas]
    target.append(temp)
    index += 1
    
y = np.asarray(target)
y_train = y.astype('float32')

y_train = np.reshape(y_train, (y_train.shape[0], y_train.shape[1] * y_train.shape[2]))

print(y_train.shape)

(1690, 144) (725, 144)


In [13]:
neurons = 512        
activation_function = 'tanh'  
loss = 'mse'                  
optimizer = 'sgd'          
dropout = 0.25                
batch_size = 100       
epochs = 1000        
# window_len = 30                     

In [14]:
def build_model(inputs, output_size, neurons, activ_func=activation_function, dropout=dropout, loss=loss, optimizer=optimizer):
    model = Sequential()
    model.add(LSTM(neurons, return_sequences=True, input_shape=(inputs.shape[1], inputs.shape[2]), activation=activ_func))
    model.add(Dropout(dropout))
    model.add(LSTM(neurons, return_sequences=True, activation=activ_func))
    model.add(Dropout(dropout))
    model.add(LSTM(neurons, activation=activ_func))
    model.add(Dropout(dropout))
    model.add(Dense(units=output_size))
    model.add(Activation(activ_func))
    model.compile(loss=loss, optimizer=optimizer, metrics=['acc','mae'])
    model.summary()
    return model

In [15]:
model = build_model(x_train, output_size=144, neurons=neurons)
# load model

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 36, 512)           1296384   
_________________________________________________________________
dropout_1 (Dropout)          (None, 36, 512)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 36, 512)           2099200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 36, 512)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 144)               73872     
__________

In [16]:
model_history = model.fit(x_train, y_train, epochs = epochs, batch_size = batch_size, verbose= 1)
model.save('LSTM_ma_4in4out.h5')
# 10 times: _ times done

Epoch 1/10000
1690/1690 [==============================] - 6s 4ms/step - loss: 30.8444 - acc: 0.0219 - mean_absolute_error: 2.5372
Epoch 2/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 30.3914 - acc: 0.0408 - mean_absolute_error: 2.5155
Epoch 3/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 29.8949 - acc: 0.0574 - mean_absolute_error: 2.4893
Epoch 4/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 29.4807 - acc: 0.0491 - mean_absolute_error: 2.4636
Epoch 5/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 29.1736 - acc: 0.0509 - mean_absolute_error: 2.4416
Epoch 6/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 28.9667 - acc: 0.0527 - mean_absolute_error: 2.4241
Epoch 7/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 28.8208 - acc: 0.0515 - mean_absolute_error: 2.4109
Epoch 8/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 28.7

Epoch 125/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 28.0119 - acc: 0.0485 - mean_absolute_error: 2.3308
Epoch 126/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 28.0053 - acc: 0.0538 - mean_absolute_error: 2.3302
Epoch 127/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 28.0102 - acc: 0.0544 - mean_absolute_error: 2.3310
Epoch 128/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 28.0046 - acc: 0.0479 - mean_absolute_error: 2.3303
Epoch 129/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 27.9962 - acc: 0.0675 - mean_absolute_error: 2.3286
Epoch 130/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 27.9931 - acc: 0.0467 - mean_absolute_error: 2.3285
Epoch 131/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 27.9953 - acc: 0.0491 - mean_absolute_error: 2.3283
Epoch 132/10000
1690/1690 [==============================] - 3s 2ms/s

1690/1690 [==============================] - 3s 2ms/step - loss: 27.7615 - acc: 0.0686 - mean_absolute_error: 2.3070
Epoch 248/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 27.7537 - acc: 0.0728 - mean_absolute_error: 2.3061
Epoch 249/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 27.7368 - acc: 0.0692 - mean_absolute_error: 2.3039
Epoch 250/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 27.7427 - acc: 0.0811 - mean_absolute_error: 2.3043
Epoch 251/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 27.7420 - acc: 0.0722 - mean_absolute_error: 2.3044
Epoch 252/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 27.7483 - acc: 0.0811 - mean_absolute_error: 2.3055
Epoch 253/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 27.7283 - acc: 0.0669 - mean_absolute_error: 2.3031
Epoch 254/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 27.7

1690/1690 [==============================] - 3s 2ms/step - loss: 27.5723 - acc: 0.0763 - mean_absolute_error: 2.2885
Epoch 370/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 27.5733 - acc: 0.0846 - mean_absolute_error: 2.2884
Epoch 371/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 27.5681 - acc: 0.0751 - mean_absolute_error: 2.2876
Epoch 372/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 27.5625 - acc: 0.0751 - mean_absolute_error: 2.2867
Epoch 373/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 27.5655 - acc: 0.0734 - mean_absolute_error: 2.2872
Epoch 374/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 27.5573 - acc: 0.0692 - mean_absolute_error: 2.2860
Epoch 375/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 27.5641 - acc: 0.0817 - mean_absolute_error: 2.2874
Epoch 376/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 27.5

1690/1690 [==============================] - 3s 2ms/step - loss: 27.4779 - acc: 0.0716 - mean_absolute_error: 2.2741
Epoch 492/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 27.4789 - acc: 0.0781 - mean_absolute_error: 2.2739
Epoch 493/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 27.4903 - acc: 0.0840 - mean_absolute_error: 2.2744
Epoch 494/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 27.5640 - acc: 0.0669 - mean_absolute_error: 2.2831
Epoch 495/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 27.4971 - acc: 0.0704 - mean_absolute_error: 2.2761
Epoch 496/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 27.4753 - acc: 0.0734 - mean_absolute_error: 2.2737
Epoch 497/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 27.4715 - acc: 0.0716 - mean_absolute_error: 2.2732
Epoch 498/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 27.4

...

1690/1690 [==============================] - 3s 2ms/step - loss: 26.7359 - acc: 0.1574 - mean_absolute_error: 2.0152
Epoch 9398/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 26.7359 - acc: 0.1639 - mean_absolute_error: 2.0150
Epoch 9399/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 26.7361 - acc: 0.1479 - mean_absolute_error: 2.0153
Epoch 9400/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 26.7354 - acc: 0.1621 - mean_absolute_error: 2.0146
Epoch 9401/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 26.7358 - acc: 0.1515 - mean_absolute_error: 2.0151
Epoch 9402/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 26.7360 - acc: 0.1538 - mean_absolute_error: 2.0151
Epoch 9403/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 26.7359 - acc: 0.1533 - mean_absolute_error: 2.0150
Epoch 9404/10000
1690/1690 [==============================] - 3s 2ms/step - los

1690/1690 [==============================] - 3s 2ms/step - loss: 26.7348 - acc: 0.1604 - mean_absolute_error: 2.0141
Epoch 9520/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 26.7354 - acc: 0.1609 - mean_absolute_error: 2.0146
Epoch 9521/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 26.7353 - acc: 0.1479 - mean_absolute_error: 2.0148
Epoch 9522/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 26.7352 - acc: 0.1598 - mean_absolute_error: 2.0146
Epoch 9523/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 26.7356 - acc: 0.1651 - mean_absolute_error: 2.0148
Epoch 9524/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 26.7349 - acc: 0.1680 - mean_absolute_error: 2.0144
Epoch 9525/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 26.7353 - acc: 0.1485 - mean_absolute_error: 2.0144
Epoch 9526/10000
1690/1690 [==============================] - 3s 2ms/step - los

1690/1690 [==============================] - 3s 2ms/step - loss: 26.7347 - acc: 0.1663 - mean_absolute_error: 2.0139
Epoch 9642/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 26.7348 - acc: 0.1550 - mean_absolute_error: 2.0139
Epoch 9643/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 26.7346 - acc: 0.1509 - mean_absolute_error: 2.0137
Epoch 9644/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 26.7347 - acc: 0.1592 - mean_absolute_error: 2.0143
Epoch 9645/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 26.7346 - acc: 0.1657 - mean_absolute_error: 2.0139
Epoch 9646/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 26.7351 - acc: 0.1716 - mean_absolute_error: 2.0142
Epoch 9647/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 26.7353 - acc: 0.1663 - mean_absolute_error: 2.0146
Epoch 9648/10000
1690/1690 [==============================] - 3s 2ms/step - los

1690/1690 [==============================] - 3s 2ms/step - loss: 26.7344 - acc: 0.1550 - mean_absolute_error: 2.0137
Epoch 9764/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 26.7348 - acc: 0.1562 - mean_absolute_error: 2.0139
Epoch 9765/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 26.7348 - acc: 0.1669 - mean_absolute_error: 2.0139
Epoch 9766/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 26.7339 - acc: 0.1651 - mean_absolute_error: 2.0131
Epoch 9767/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 26.7349 - acc: 0.1598 - mean_absolute_error: 2.0141
Epoch 9768/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 26.7344 - acc: 0.1467 - mean_absolute_error: 2.0134
Epoch 9769/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 26.7345 - acc: 0.1604 - mean_absolute_error: 2.0136
Epoch 9770/10000
1690/1690 [==============================] - 3s 2ms/step - los

1690/1690 [==============================] - 3s 2ms/step - loss: 26.7339 - acc: 0.1621 - mean_absolute_error: 2.0129
Epoch 9886/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 26.7344 - acc: 0.1663 - mean_absolute_error: 2.0135
Epoch 9887/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 26.7342 - acc: 0.1746 - mean_absolute_error: 2.0133
Epoch 9888/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 26.7342 - acc: 0.1669 - mean_absolute_error: 2.0130
Epoch 9889/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 26.7338 - acc: 0.1598 - mean_absolute_error: 2.0129
Epoch 9890/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 26.7341 - acc: 0.1639 - mean_absolute_error: 2.0129
Epoch 9891/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 26.7342 - acc: 0.1586 - mean_absolute_error: 2.0132
Epoch 9892/10000
1690/1690 [==============================] - 3s 2ms/step - los